<a href="https://colab.research.google.com/github/dkimds/HF-Transformer/blob/main/Llama2_fine_tuning_korquad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama 2를 KorQuad 데이터셋에 맞게 fine-tuning 하기
### References:
1.   https://www.youtube.com/watch?v=LslC2nKEEGU
2.   https://colab.research.google.com/drive/1JDnGJbxT8fSqwnXY8J-XFo73AtiSuQMe?usp=sharing

In [1]:
!nvidia-smi

Tue May 13 10:07:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## AutoTrain Advanced
https://github.com/huggingface/autotrain-advanced

In [2]:
!pip install -q autotrain-advanced

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 53.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.3/341.3 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.9/269.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 13.4 MB/s eta 0

## PyTorch 업데이트

In [3]:
!autotrain setup --update-torch

INFO     | 2025-05-13 10:10:30 | autotrain.cli.run_setup:run:43 - Installing latest xformers
INFO     | 2025-05-13 10:10:31 | autotrain.cli.run_setup:run:45 - Successfully installed latest xformers
INFO     | 2025-05-13 10:10:31 | autotrain.cli.run_setup:run:51 - Installing latest PyTorch
INFO     | 2025-05-13 10:10:34 | autotrain.cli.run_setup:run:53 - Successfully installed latest PyTorch


In [ ]:
!pip install triton
!pip uninstall bitsandbytes -y
!pip install bitsandbytes

## Korquad 데이터셋에 맞게 Llama2 Fine-Tuning

- autotrain 설정값: https://github.com/huggingface/autotrain-advanced/blob/f1367b590dfc53d240e9684779991da540590386/src/autotrain/cli/run_llm.py#L21 (**과거 버전[0.6.35]**)
- autotrain 설정값: https://github.com/huggingface/autotrain-advanced/blob/main/src/autotrain/cli/run_llm.py#L17 (**최신 버전[0.6.80]**)


In [ ]:
# Data Augmentation 적용 x
# !autotrain llm --train \
#     --project_name "llama2-korquad-finetuning" \
#     --model "TinyPixel/Llama-2-7B-bf16-sharded" \
#     --data_path "korquad_prompt" \
#     --text_column "text" \
#     --use_peft \
#     --use_int4 \
#     --learning_rate 2e-4 \
#     --train_batch_size 4 \
#     --num_train_epochs 100 \
#     --trainer sft \
#     --model_max_length 256
# Data Augmentation 적용 o (과거 버전)[0.6.35]
# !autotrain llm --train \
#     --project_name "llama2-korquad-finetuning-da" \
#     --model "TinyPixel/Llama-2-7B-bf16-sharded" \
#     --data_path "korquad_prompt_da" \
#     --text_column "text" \
#     --use_peft \
#     --use_int4 \
#     --learning_rate 2e-4 \
#     --train_batch_size 8 \
#     --num_train_epochs 40 \
#     --trainer sft \
#     --model_max_length 256
# Data Augmentation 적용 o (최신 버전)[0.6.80]
!autotrain llm --train \
    --project-name "llama2-korquad-finetuning-da" \
    --model "TinyPixel/Llama-2-7B-bf16-sharded" \
    --data-path "korquad_prompt_da" \
    --text-column "text" \
    --peft \
    --quantization "int4" \
    --lr 2e-4 \
    --batch-size 8 \
    --epochs 40 \
    --trainer sft \
    --model_max_length 256

INFO     | 2025-05-13 10:34:55 | autotrain.cli.run_llm:run:136 - Running LLM
WARNING  | 2025-05-13 10:34:55 | autotrain.trainers.common:__init__:286 - Parameters supplied but not used: backend, deploy, version, train, config, inference, func
INFO     | 2025-05-13 10:34:55 | autotrain.backends.local:create:20 - Starting local training...
INFO     | 2025-05-13 10:34:55 | autotrain.commands:launch_command:514 - ['accelerate', 'launch', '--num_machines', '1', '--num_processes', '1', '--mixed_precision', 'no', '-m', 'autotrain.trainers.clm', '--training_config', 'llama2-korquad-finetuning-da/training_params.json']
INFO     | 2025-05-13 10:34:55 | autotrain.commands:launch_command:515 - {'model': 'TinyPixel/Llama-2-7B-bf16-sharded', 'project_name': 'llama2-korquad-finetuning-da', 'data_path': 'korquad_prompt_da', 'train_split': 'train', 'valid_split': None, 'add_eos_token': True, 'block_size': 1024, 'model_max_length': 256, 'padding': 'right', 'trainer': 'sft', 'use_flash_attention_2': False

# 학습결과 zip 파일로 압축후 다운로드

In [ ]:
import zipfile
import shutil
from google.colab import files

# 압축할 폴더 이름
#folder_name = "llama2-korquad-finetuning"    # Data Augmentation 적용 x
folder_name = "llama2-korquad-finetuning-da"  # Data Augmentation 적용 o

# 생성될 ZIP 파일 이름
#zip_file_name = "llama2-korquad-finetuning.zip"  # Data Augmentation 적용 x
zip_file_name = "llama2-korquad-finetuning-da.zip" # Data Augmentation 적용 o

# 폴더를 ZIP 파일로 압축
shutil.make_archive(zip_file_name[:-4], 'zip', folder_name)

# ZIP 파일을 로컬로 다운로드
files.download(zip_file_name)